# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [1]:
import pandas as pd
import numpy as np

---
## 2. Load data

In [2]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [3]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [4]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [5]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


## Average % price changes across periods

In [6]:
# Aggregate average prices per period by company
mean_prices = price_df.groupby(['id']).agg({
    'price_off_peak_var': 'mean', 
    'price_peak_var': 'mean', 
    'price_mid_peak_var': 'mean',
    'price_off_peak_fix': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_fix': 'mean'    
}).reset_index()

# Calculate the percentage change between consecutive periods
mean_prices['off_peak_peak_var_pct_change'] = (
    (mean_prices['price_peak_var'] - mean_prices['price_off_peak_var']) / mean_prices['price_off_peak_var']
) * 100

mean_prices['peak_mid_peak_var_pct_change'] = (
    (mean_prices['price_mid_peak_var'] - mean_prices['price_peak_var']) / mean_prices['price_peak_var']
) * 100

mean_prices['off_peak_mid_peak_var_pct_change'] = (
    (mean_prices['price_mid_peak_var'] - mean_prices['price_off_peak_var']) / mean_prices['price_off_peak_var']
) * 100

mean_prices['off_peak_peak_fix_pct_change'] = (
    (mean_prices['price_peak_fix'] - mean_prices['price_off_peak_fix']) / mean_prices['price_off_peak_fix']
) * 100

mean_prices['peak_mid_peak_fix_pct_change'] = (
    (mean_prices['price_mid_peak_fix'] - mean_prices['price_peak_fix']) / mean_prices['price_peak_fix']
) * 100

mean_prices['off_peak_mid_peak_fix_pct_change'] = (
    (mean_prices['price_mid_peak_fix'] - mean_prices['price_off_peak_fix']) / mean_prices['price_off_peak_fix']
) * 100

print(mean_prices)

                                     id  price_off_peak_var  price_peak_var  \
0      0002203ffbb812588b632b9e628cc38d            0.124338        0.103794   
1      0004351ebdd665e6ee664792efc4fd13            0.146426        0.000000   
2      0010bcc39e42b3c2131ed2ce55246e3c            0.181558        0.000000   
3      0010ee3855fdea87602a5b7aba8e42de            0.118757        0.098292   
4      00114d74e963e47177db89bc70108537            0.147926        0.000000   
...                                 ...                 ...             ...   
16091  ffef185810e44254c3a4c6395e6b4d8a            0.138863        0.115125   
16092  fffac626da707b1b5ab11e8431a4d0a2            0.147137        0.000000   
16093  fffc0cacd305dd51f316424bbb08d1bd            0.153879        0.129497   
16094  fffe4f5646aa39c7f97f95ae2679ce64            0.123858        0.103499   
16095  ffff7fa066f1fb305ae285bb03bf325a            0.125360        0.104895   

       price_mid_peak_var  price_off_peak_fix  pric

Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

## Contract Duration

In [7]:
# Converting to pandas DataFrame
df = pd.DataFrame(df)

# Converting the 'date_activ' and 'date_end' columns to datetime
df['date_activ'] = pd.to_datetime(df['date_activ'])
df['date_end'] = pd.to_datetime(df['date_end'])

# Computing contract duration in days
df['contract_duration_days'] = (df['date_end'] - df['date_activ']).dt.days

# Displaying result
print(df[['id', 'contract_duration_days']])

                                     id  contract_duration_days
0      24011ae4ebbe3035111d65fa7c15bc57                    1096
1      d29c2c54acc38ff3c0614d0a653813dd                    2566
2      764c75f661154dac3a6c254cd082ea7d                    2192
3      bba03439a292a1e166f80264c16191cb                    2192
4      149d57cf92fc41cf94415803a877cb4b                    2245
...                                 ...                     ...
14601  18463073fb097fc0ac5d3e040f356987                    1445
14602  d0a6f71671571ed83b2645d23af6de00                    1461
14603  10e6828ddd62cbcf687cb74928c4c2d2                    1460
14604  1cf20fd6206d7678d5bcafd28c53b4db                    1461
14605  563dde550fd624d7352f3de77c0cdfcd                    2556

[14606 rows x 2 columns]


## Extract day, month, year and Quarter from dates

In [8]:
# Extract the quarter from the date column
df['quarter_activ'] = df['date_activ'].dt.quarter
df['quarter_end'] = df['date_end'].dt.quarter
df['quarter_renewal'] = df['date_renewal'].dt.quarter
df['quarter_modif_prod'] = df['date_modif_prod'].dt.quarter

# Extract the month from the date column
df['month_activ'] = df['date_activ'].dt.month
df['month_end'] = df['date_end'].dt.month
df['month_renewal'] = df['date_renewal'].dt.month
df['month_modif_prod'] = df['date_modif_prod'].dt.month

# Extract the year from the date column
df['year_activ'] = df['date_activ'].dt.year
df['year_end'] = df['date_end'].dt.year
df['year_renewal'] = df['date_renewal'].dt.year
df['year_modif_prod'] = df['date_modif_prod'].dt.year

# Extract the day from the date column
df['day_activ'] = df['date_activ'].dt.day
df['day_end'] = df['date_end'].dt.day
df['day_renewal'] = df['date_renewal'].dt.day
df['day_modif_prod'] = df['date_modif_prod'].dt.day

# Print results
print(df[['quarter_activ', 'quarter_end', 'quarter_renewal', 'quarter_modif_prod', 'month_activ', 'month_end', 'month_renewal', 'month_modif_prod', 'year_activ', 'year_end', 'year_renewal', 'year_modif_prod', 'day_activ', 'day_end', 'day_renewal', 'day_modif_prod']].head())

   quarter_activ  quarter_end  quarter_renewal  quarter_modif_prod  \
0              2            2                2                   4   
1              3            3                3                   3   
2              2            2                2                   2   
3              1            1                1                   1   
4              1            1                1                   1   

   month_activ  month_end  month_renewal  month_modif_prod  year_activ  \
0            6          6              6                11        2013   
1            8          8              8                 8        2009   
2            4          4              4                 4        2010   
3            3          3              3                 3        2010   
4            1          3              3                 1        2010   

   year_end  year_renewal  year_modif_prod  day_activ  day_end  day_renewal  \
0      2016          2015             2015         15  

## Loyalty indicators

In [9]:
# Create binary feature based on the 2 year threshold
threshold = 2
df['has_been_with_company_more_than_2_years'] = (df['num_years_antig'] > threshold).astype(int)

# Alternatively, for a 5-year threshold
threshold_5_years = 5
df['has_been_with_company_more_than_5_years'] = (df['num_years_antig'] > threshold_5_years).astype(int)

# Print results
print(df[['year_activ', 'year_end', 'num_years_antig', 'has_been_with_company_more_than_2_years', 'has_been_with_company_more_than_5_years']].head())

   year_activ  year_end  num_years_antig  \
0        2013      2016                3   
1        2009      2016                6   
2        2010      2016                6   
3        2010      2016                6   
4        2010      2016                6   

   has_been_with_company_more_than_2_years  \
0                                        1   
1                                        1   
2                                        1   
3                                        1   
4                                        1   

   has_been_with_company_more_than_5_years  
0                                        0  
1                                        1  
2                                        1  
3                                        1  
4                                        1  


In [10]:
# Contract duration in years
df['contract_duration_years'] = df['contract_duration_days'] / 365.25
df['contract_duration_years'].median()

# Example of creating a 'contract_renewed' column
df['contract_renewed'] = df['date_renewal'].notna().astype(int)

# Flag indicating if the contract was renewed and duration was over 5 years (median value of contract duration)
df['renewed_and_long_term'] = (df['contract_renewed'] == 1) & (df['contract_duration_years'] > 5)
df['renewed_and_long_term'] = df['renewed_and_long_term'].astype(int)

print(df[['contract_duration_years', 'renewed_and_long_term']])

       contract_duration_years  renewed_and_long_term
0                     3.000684                      0
1                     7.025325                      1
2                     6.001369                      1
3                     6.001369                      1
4                     6.146475                      1
...                        ...                    ...
14601                 3.956194                      0
14602                 4.000000                      0
14603                 3.997262                      0
14604                 4.000000                      0
14605                 6.997947                      1

[14606 rows x 2 columns]


## Revenue per product

In [11]:
# Calculate the average net margin per product
df['avg_net_margin_per_product'] = df['net_margin'] / df['nb_prod_act']

# Handle cases where 'nb_prod_act' is 0 or NaN to avoid division by zero
df['avg_net_margin_per_product'] = df['avg_net_margin_per_product'].replace([float('inf'), -float('inf')], pd.NA)

# Print results
print(df[['net_margin', 'nb_prod_act', 'avg_net_margin_per_product']].head(3))

   net_margin  nb_prod_act  avg_net_margin_per_product
0      678.99            2                     339.495
1       18.89            1                      18.890
2        6.60            1                       6.600


## Contract Renewed

In [12]:
# Print the first few rows to check the results
print(df[['date_renewal', 'date_end', 'contract_renewed', 'churn']].head())

  date_renewal   date_end  contract_renewed  churn
0   2015-06-23 2016-06-15                 1      1
1   2015-08-31 2016-08-30                 1      0
2   2015-04-17 2016-04-16                 1      0
3   2015-03-31 2016-03-30                 1      0
4   2015-03-09 2016-03-07                 1      0


Transforming Variables

In [13]:
# Transforing Boolean variables to int
df['has_gas'] = df['has_gas'].replace(['t', 'f'], [1, 0])

# Transform into categorical type
df['channel_sales'] = df['channel_sales'].astype('category')

# Transform into categorical type
df = pd.get_dummies(df, columns=['channel_sales'], prefix='channel')
df = df.drop(columns=['channel_sddiedcslfslkckwlfkdpoeeailfpeds', 'channel_epumfxlbckeskwekxbiuasklxalciiuu', 'channel_fixdbufsefwooaasfcxdxadsiekoceaa'])

# Transform into categorical type
df['origin_up'] = df['origin_up'].astype('category')
df = pd.get_dummies(df, columns=['origin_up'], prefix='origin_up')
df = df.drop(columns=['origin_up_MISSING', 'origin_up_usapbepcfoloekilkwsdiboslwaxobdp', 'origin_up_ewxeelcelemmiwuafmddpobolfuxioce'])
df.head()

C:\Users\gadas\AppData\Local\Temp\ipykernel_33836\474870689.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['has_gas'] = df['has_gas'].replace(['t', 'f'], [1, 0])


,id,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,...,renewed_and_long_term,avg_net_margin_per_product,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,...,0,339.495,False,False,True,False,False,False,False,True
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,...,1,18.890,True,False,False,False,False,True,False,False
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,...,1,6.600,False,False,True,False,False,True,False,False
3,bba03439a292a1e166f80264c16191cb,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,...,1,25.460,False,False,False,True,False,True,False,False
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,...,1,47.980,True,False,False,False,False,True,False,False


Standardize Numerical variables

In [14]:
# Apply log10 transformation
df["cons_12m"] = np.log10(df["cons_12m"] + 1)
df["cons_gas_12m"] = np.log10(df["cons_gas_12m"] + 1)
df["cons_last_month"] = np.log10(df["cons_last_month"] + 1)
df["forecast_cons_12m"] = np.log10(df["forecast_cons_12m"] + 1)
df["forecast_cons_year"] = np.log10(df["forecast_cons_year"] + 1)
df["forecast_meter_rent_12m"] = np.log10(df["forecast_meter_rent_12m"] + 1)
df["imp_cons"] = np.log10(df["imp_cons"] + 1)

In [15]:
# Columns to delete
columns_to_delete = ['date_renewal', 'date_end', 'date_activ', 'date_modif_prod', 'contract_renewed', 'num_years_antig', 'contract_duration_years', 'forecast_cons_year']

# Drop the specified columns
df = df.drop(columns=columns_to_delete)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 65 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          14606 non-null  object 
 1   cons_12m                                    14606 non-null  float64
 2   cons_gas_12m                                14606 non-null  float64
 3   cons_last_month                             14606 non-null  float64
 4   forecast_cons_12m                           14606 non-null  float64
 5   forecast_discount_energy                    14606 non-null  float64
 6   forecast_meter_rent_12m                     14606 non-null  float64
 7   forecast_price_energy_off_peak              14606 non-null  float64
 8   forecast_price_energy_peak                  14606 non-null  float64
 9   forecast_price_pow_off_peak                 14606 non-null  float64
 10  has_gas   